# FugueSQL and Python

`FugueSQL` is integrates with Python by creating extensions and applying them in the `FugueSQLWorkflow`. This tutorial will show an example of applying a [Fugue Transformer](../transformer.ipynb). Additionally, we'll see the simpler ways that the Python interface and `FugueSQL` interact with each other.

In [ ]:
from fugue_notebook import setup
setup()

In [ ]:
import pandas as pd
df = pd.DataFrame({"a": [0,1], "b": ["hello", "world"]},"a:int,b:str")

## [Jinja](https://jinja.palletsprojects.com/) Templating to Insert Variables

Before going to functions, the simplest way `FugueSQL` integrates with Python is through [Jinja](https://jinja.palletsprojects.com/) templating. `DataFrames` defined previously are automatically accessible by the `DAG`. Variables on the other hand, need to be passed with [Jinja](https://jinja.palletsprojects.com/) templating.

In [ ]:
x=0

In [ ]:
%%fsql
SELECT * FROM df WHERE a={{x}}  # see we can use variable x directly
PRINT

Jinja templating allows for flexible construction of queries such as using for loops:

In [ ]:
with FugueSQLWorkflow() as dag:
    tbl = dag.df([[0,"x"],[1,"y"],[2,"z"],[0,"a"]], "a:int, b:str")
    values = {'b': ['x','y','z']}
    fsql("""SELECT * FROM tbl WHERE b IN ({%- for i in b -%}
    {%- if loop.index0 < loop.length - 1 -%}'{{-i-}}',
    {%- else -%}'{{-i-}}'
    {%- endif -%}
    {%- endfor -%})
    PRINT """, values).run()


## Using [Transformers](../transformer.ipynb)

`Fugue` has different [extensions](extensions.ipynb) that allow Python to interact with SQL. `Transformer` is most commonly used because it modifies that data in a dataframe. Below we create a `Transformer` in Python and apply it in the `FugueSQLWorkflow`. More on `Transformer` syntax can be found here.

In [ ]:
import re
from typing import Iterable, Dict, Any

# schema: *, vowel_count:int, consonant_count:int
def letter_count(df:Iterable[Dict[str,Any]]) -> Iterable[Dict[str,Any]]:
    for row in df:
        row['vowel_count'] = len(re.findall(r'[aeiou]', row['word'], flags=re.IGNORECASE))
        space_count = len(re.findall(r'[ -]', row['word'], flags=re.IGNORECASE))
        row['consonant_count'] = len(row['word']) - row['vowel_count'] - space_count
        yield row

with FugueSQLWorkflow() as dag:
    df = dag.df([[0,"hello"],[1,"world"]],"number:int,word:str")
    dag("""
    SELECT * 
    FROM df 
    WHERE number=1
    TRANSFORM USING letter_count
    PRINT
    """)